# RAG + Vector DB + Pinecone (notater + kode-maler)

Dette er en “bachelor-/produksjonsvennlig” samling notater du kan bruke i rapport og implementasjon.

**Tema:**

- når du trenger RAG + vector DB
- chunking/splitting (context window)
- embeddings + vector space (cosine/dot/euclid)
- retrieval: similarity vs MMR
- stuffing (stuff chain)
- Pinecone (managed vector DB)
- mini-arkitektur (klar til rapport)


## 1) Hva prøver du å løse med RAG + Vector DB?

Tenk på to typer spørsmål:

1. **Eksakte spørsmål (SQL/NoSQL passer)**
   - “Finn ordre med `id=123`”
   - “Hent kunden med e-post X”

2. **Semantiske spørsmål (Vector DB passer)**
   - “Hva sier runbooken om *high CPU*?”
   - “Hvordan håndterer vi chargeback/dispute?”

**RAG** bygger bro:

- Vector DB finner relevante tekstbiter (**retrieval**)
- LLM bruker dem som kontekst for å svare (**generation**)


## 2) Loading + splitting (chunking)

### Hvorfor splitting?

LLM-er har et **kontekstvindu** (maks tokens per forespørsel). Du kan ikke sende hele PDF/SLA/runbook i én prompt.

Du deler derfor inn i **chunks** som:

- er små nok til å passe inn
- er store nok til å gi mening
- har **overlap** slik at kontekst ikke “knekker”

### Chunking styres av

- **chunk_size** (f.eks. 1500 tegn)
- **chunk_overlap** (f.eks. 500 tegn)
- **separators** (f.eks. `\n\n`, `\n`, `. `, ` `)

### Splittere (praktisk)

- `CharacterTextSplitter`: enkel fast oppdeling
- `RecursiveCharacterTextSplitter`: prøver snille grenser først (ofte best default)
- `TokenTextSplitter`: splitter på tokens (mer presist mot kontekstvindu)
- `MarkdownHeaderTextSplitter`: splitter på `# / ## / ###` (perfekt for notater)

### Metadata = gull i bachelor

Lag chunks som `Document` med:

- `page_content`
- `metadata` (tittel, side, filnavn, URL)

Da kan du skrive: “Hentet fra Lecture X / side Y”.


## 3) Embeddings og vector space (intuition)

### Hva er embedding?

En **embedding** er en vektor (liste med tall) som representerer “meningen” i en tekstbit.

Du kan formulere det slik:

> Embeddings projiserer abstrakte konsepter inn i et kontinuerlig numerisk rom der semantisk likhet kan måles med avstand/similaritet.

### Likhet/avstand

- **Cosine similarity**: vinkelen mellom vektorer (retning ≈ mening)
- **Dot product**: retning + magnitude (raskt; hvis vektorer normaliseres ≈ cosine)
- **Euclidean distance**: “luftlinje-avstand”
- **Manhattan distance**: “rutenett-avstand” (sjeldnere brukt i tekst-embeddings)


## 4) SQL vs NoSQL vs Vector DB (rapport-linje)

- **SQL**: eksakt match + relasjoner + transaksjoner
- **NoSQL**: fleksible dokumenter/key-value
- **Vector DB**: semantisk match (nærmeste nabo i embedding-rom), ofte med metadata-filter

Paste-ready setning:

> SQL/NoSQL er optimalisert for eksakt matching og strukturerte spørringer, mens vektordatabaser er optimalisert for semantisk matching der likhet beregnes i et embedding-rom.


## 5) Retrieval: similarity search vs MMR

### Similarity search

Henter top-k mest like chunks.

- Ulempe: du kan få duplikater/nesten-like chunks (støy)

### MMR (Max Marginal Relevance)

Henter chunks som er:

- relevante **og**
- mer **diverse**

Eksempel (SLA/logistikk):

- similarity: 5 nesten identiske “delay”-avsnitt
- MMR: delay + escalation + compensation + frister


## 6) Stuffing (“stuff chain”)

**Stuffing** betyr: hentede chunks settes direkte inn i prompt-konteksten.

Fordeler:

- enklest å implementere
- lett å forklare i rapport

Ulemper (akademisk viktig):

- kan sprenge kontekstvindu
- hvis retrieval gir støy, “stuffer” du støy

Paste-ready setning:

> I denne implementasjonen brukes en “stuffing”-strategi der top-k hentede chunks legges direkte i promptkonteksten. Dette forenkler arkitekturen, men krever stram kontroll på chunk-størrelse, overlap og antall dokumenter for å unngå kontekstovertrekk og irrelevante svar.


## 7) Pinecone (hva og hvorfor)

Pinecone er en managed vector database som lar deg:

- opprette indeks (dimension + metric, ofte cosine)
- upsert embeddings + metadata
- query med semantisk søk + metadata-filter

Brukes når du vil ha:

- skalerbar og driftbar vector store
- rask ANN-søk
- mindre “DIY” enn lokal Chroma


In [ ]:
# Kode-mal: RAG med LangChain + Chroma (template)
# (Krever pakker: langchain/langchain-openai/langchain-community/chromadb/python-dotenv)

# import os
# from dotenv import load_dotenv
# load_dotenv()
# assert os.getenv("OPENAI_API_KEY")
#
# from langchain_community.document_loaders import TextLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain_community.vectorstores import Chroma
# from langchain_core.prompts import PromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
#
# docs = TextLoader("data/my_corpus.txt", encoding="utf-8").load()
# splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=500)
# chunks = splitter.split_documents(docs)
#
# emb = OpenAIEmbeddings(model="text-embedding-3-large")
# vectordb = Chroma.from_documents(chunks, emb, persist_directory="chroma_db")
# vectordb.persist()
#
# retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 5, "fetch_k": 20})
#
# def format_docs(docs):
#     out = []
#     for d in docs:
#         src = d.metadata.get("source", "unknown")
#         out.append(f"[{src}] {d.page_content}")
#     return "\n\n".join(out)
#
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# prompt = PromptTemplate.from_template(
#     "Du er en presis fagassistent. Svar kun basert på konteksten.\n"
#     "Hvis ikke nok kontekst: skriv 'Ikke nok informasjon i kildene'.\n\n"
#     "Spørsmål: {question}\n\nKontekst:\n{context}\n"
# )
#
# chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )
#
# print(chain.invoke("Forklar forskjellen på similarity og MMR"))


In [ ]:
# Kode-mal: Pinecone + LangChain (template)
# NB: Dette er en mal basert på notatene dine. Tilpass dimensjon/model.

# import os
# from dotenv import load_dotenv
# load_dotenv()
#
# from pinecone import Pinecone, ServerlessSpec
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain_community.document_loaders import TextLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_core.prompts import ChatPromptTemplate
#
# pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
# index_name = "my-index"
# dimension = 1536  # må matche embedding-dimensjonen
# metric = "cosine"
#
# existing = [x["name"] for x in pc.list_indexes().get("indexes", [])]
# if index_name not in existing:
#     pc.create_index(
#         name=index_name,
#         dimension=dimension,
#         metric=metric,
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )
#
# index = pc.Index(index_name)
#
# # docs -> chunks
# docs = TextLoader("data/corpus.txt", encoding="utf-8").load()
# splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=500)
# chunks = splitter.split_documents(docs)
# for d in chunks:
#     d.page_content = " ".join(d.page_content.split())
#
# emb = OpenAIEmbeddings()
# texts = [d.page_content for d in chunks]
# metas = [d.metadata for d in chunks]
# vectors = emb.embed_documents(texts)
#
# to_upsert = []
# for i, (vec, meta, text) in enumerate(zip(vectors, metas, texts)):
#     meta = dict(meta)
#     meta["text"] = text
#     to_upsert.append((f"doc-{i}", vec, meta))
#
# index.upsert(vectors=to_upsert)
#
# # query + stuffing
# q = "Forklar forskjellen på similarity search og MMR i retrieval"
# qvec = emb.embed_query(q)
# res = index.query(vector=qvec, top_k=5, include_metadata=True)
# ctx = "\n\n---\n\n".join([m["metadata"].get("text", "") for m in res["matches"]])
#
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "Svar kun basert på konteksten. Hvis ikke nok: 'Ikke nok informasjon i kildene'."),
#     ("user", "Spørsmål: {q}\n\nKontekst:\n{ctx}"),
# ])
#
# answer = llm.invoke(prompt.format_messages(q=q, ctx=ctx)).content
# print(answer)


## 8) Mini-arkitektur (klar til rapport)

**Indexing (offline):**

Load → Clean → Split → Embed → Store (Vector DB)

**Query-time (online):**

Embed query → Retrieve (Similarity/MMR) → Stuff context → Prompt → LLM → Parse/Validate → Log/Trace

Figurtekst (paste-ready):

> Figur X: RAG-pipeline modellert som graf: dokumenter splittes i overlappende chunks, embeddes til vektorer og lagres i en vektordatabase. Ved spørsmål hentes relevante chunks (similarity/MMR) og “stuffes” inn i promptkonteksten før LLM genererer et svar.


## 9) Formler (LaTeX) + Python-ekvivalenter

### Cosine similarity

\[
\cos(\theta) = \frac{\vec a \cdot \vec b}{\lVert \vec a \rVert \, \lVert \vec b \rVert}
\]

### Dot product

\[
\vec a \cdot \vec b = \sum_i a_i b_i
\]

### Euclidean distance

\[
\lVert \vec a - \vec b \rVert_2 = \sqrt{\sum_i (a_i - b_i)^2}
\]


In [ ]:
import math
from typing import Sequence


def dot(a: Sequence[float], b: Sequence[float]) -> float:
    if len(a) != len(b):
        raise ValueError("Vectors must have same length")
    return sum(x * y for x, y in zip(a, b))


def l2_norm(a: Sequence[float]) -> float:
    return math.sqrt(dot(a, a))


def cosine_similarity(a: Sequence[float], b: Sequence[float]) -> float:
    denom = l2_norm(a) * l2_norm(b)
    if denom == 0:
        return 0.0
    return dot(a, b) / denom


def euclidean_distance(a: Sequence[float], b: Sequence[float]) -> float:
    if len(a) != len(b):
        raise ValueError("Vectors must have same length")
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(a, b)))


a = [1.0, 2.0, 3.0]
b = [2.0, 0.0, 4.0]

{
    "dot": dot(a, b),
    "cosine": cosine_similarity(a, b),
    "euclidean": euclidean_distance(a, b),
}
